![USD](usd.png)

### University of San Diego 

### Master of Science, Applied Data Science 

#### Contributors

- Ebad Akhter
- Kevin Baum
- Salvador Sanchez

***

In [93]:
#Libraries
import pandas as pd
from datetime import datetime
import numpy as np
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import PasswordHolder

# Load Data

In [2]:
#function to download yearly data
def Get_Data(year):
    url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
    df = pd.read_csv(url)
    return df

In [3]:
#Current year
current_year = datetime.now().year

In [73]:
#Current Data
current_data = Get_Data(current_year)

#Last Year
last_year_data = Get_Data(current_year-1)

#Year - 2
yearM2_data = Get_Data(current_year-2)

current_data.tail()

incident_num            date_time  day_of_week  address_number_primary  \
31875  E23010034178  2023-01-24 23:40:34            3                       0   
31876  E23010034179  2023-01-24 23:42:49            3                    3800   
31877  E23010034180  2023-01-24 23:43:19            3                    1800   
31878  E23010034182  2023-01-24 23:44:36            3                    1000   
31879  E23010034185  2023-01-24 23:45:15            3                       0   

      address_dir_primary address_road_primary address_sfx_primary  \
31875                 NaN              GENESEE                 AVE   
31876                 NaN               ORANGE                 AVE   
31877                 NaN                 UPAS                  ST   
31878                 NaN                    B                  ST   
31879                 NaN              COLLEGE                 AVE   

       address_dir_intersecting address_road_intersecting  \
31875                       NaN                  APPLETON   
31876                       NaN                       NaN   
31877                       NaN                       NaN   
31878                       NaN                       NaN   
31879                       NaN                 MONTEZUMA   

       address_sfx_intersecting call_type disposition  beat  priority  
31875                       NaN         T           K   114         2  
31876                       NaN      459A         CAN   838         2  
31877                       NaN        FU           K   531         2  
31878                       NaN      1171         CAN   524         1  
31879                       NaN         T           K   327         2

# EDA

In [5]:
current_data.describe()

day_of_week  address_number_primary  address_dir_intersecting  \
count  30350.000000            30350.000000                       0.0   
mean       3.892389             3381.046787                       NaN   
std        1.984939             3384.459204                       NaN   
min        1.000000                0.000000                       NaN   
25%        2.000000              600.000000                       NaN   
50%        4.000000             2800.000000                       NaN   
75%        6.000000             4800.000000                       NaN   
max        7.000000            53500.000000                       NaN   

       address_sfx_intersecting          beat      priority  
count                       0.0  30350.000000  30350.000000  
mean                        NaN    492.391993      2.103295  
std                         NaN    238.326237      1.213443  
min                         NaN     -1.000000      0.000000  
25%                         NaN    313.000000      1.000000  
50%                         NaN    521.000000      2.000000  
75%                         NaN    627.000000      3.000000  
max                         NaN    937.000000      9.000000

In [6]:
last_year_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499256 entries, 0 to 499255
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   incident_num               499256 non-null  object 
 1   date_time                  499256 non-null  object 
 2   day_of_week                499256 non-null  int64  
 3   address_number_primary     499256 non-null  int64  
 4   address_dir_primary        23135 non-null   object 
 5   address_road_primary       499187 non-null  object 
 6   address_sfx_primary        455019 non-null  object 
 7   address_dir_intersecting   0 non-null       float64
 8   address_road_intersecting  85382 non-null   object 
 9   address_sfx_intersecting   0 non-null       float64
 10  call_type                  498640 non-null  object 
 11  disposition                497161 non-null  object 
 12  beat                       499256 non-null  int64  
 13  priority                   49

In [70]:
#Expore Data
def Explore_Date(df):
    #Clean Data
    df = df.dropna(how='all', axis=1)
    
    #Initial table
    freqDF = pd.DataFrame(columns=['Feature',
                                   'Mode',
                                   'Mode Freq.',
                                   'Mode %',
                                   '2nd Mode',
                                   '2nd Mode Freq.',
                                   '2nd Mode %'])
    for col in df.columns:
        freq = df[col].value_counts()
        freqdf = freq.to_frame()
        fRow = freqdf.iloc[0]
        secRow = freqdf.iloc[1]
        fPrct = fRow[0] / len(df[col])
        secPrct = secRow[0] / len(df[col])
        try:
            mode1 = int(fRow.name)
        except:
            mode1 = fRow.name
        try:
            mode2 = int(secRow.name)
        except:
            mode2 = secRow.name
        data = {'Feature':col,
                'Mode':mode1,
                'Mode Freq.':fRow[0],
                'Mode %':fPrct,\
                '2nd Mode':mode2,
                '2nd Mode Freq.':secRow[0],
                '2nd Mode %':secPrct}
        freqDF.loc[len(freqDF)] = data

    freqDF = freqDF.set_index('Feature')

    #Nulls, Counts, Cardinality
    NUllFeatures = round(df.isnull().sum() / df.shape[0],4)\
          .sort_values(ascending=False)
    Count = df.count()
    uni = df.nunique()

    #Formating
    NUllFeatures.to_frame(name="% Miss.")
    Count.to_frame(name="Count")
    uni.to_frame()
    result = pd.concat([Count, NUllFeatures,uni], axis=1)
    result.columns =["Count","% Miss.","Card."]
    result = pd.concat([result, freqDF], axis=1)
    result = result.style.format({'% Miss.': "{:.1%}",
                         'Mode %': "{:.0%}",
                         '2nd Mode %': "{:.0%}",
                         'Count': "{:,}",
                         'Card.': "{:,}",
                         'Mode Freq.': "{:,}",
                        '2nd Mode Freq.': "{:,}"})
    return result

In [75]:
Explore_Date(current_data)

In [72]:
Explore_Date(last_year_data)

In [74]:
Explore_Date(yearM2_data)

# Data Prep

In [9]:
def AddressNumberStr(df):
    df['address_number_primary_str'] = df['address_number_primary'].astype(str)
    df.address_number_primary_str.replace('0', np.nan, inplace=True)
    return df

In [22]:
def AddressField(df,City,State):
    df['Address'] = df[['address_number_primary_str',
                        'address_dir_primary',
                        'address_road_primary',
                        'address_sfx_primary']].apply(lambda x: ' '.join(x.dropna()), axis=1)
    df['Address'] = df['Address'] + ' ' + City +', ' + State
    return df

In [27]:
Address_Data = AddressNumberStr(current_data)
Address_Data = AddressField(current_data,'San Diego','California')

In [30]:
Address_Data.tail()

incident_num            date_time  day_of_week  address_number_primary  \
30345  E23010032562  2023-01-23 23:35:53            2                       0   
30346  E23010032563  2023-01-23 23:38:24            2                     400   
30347  E23010032565  2023-01-23 23:39:02            2                     600   
30348  E23010032567  2023-01-23 23:44:30            2                   13100   
30349  E23010032570  2023-01-23 23:49:32            2                       0   

      address_dir_primary address_road_primary address_sfx_primary  \
30345                 NaN                HOITT                  ST   
30346                 NaN   HOTEL CIRCLE SOUTH                 NaN   
30347                 NaN                 32ND                  ST   
30348                 NaN            SEA KNOLL                  CT   
30349                 NaN                    E                  ST   

       address_dir_intersecting address_road_intersecting  \
30345                       NaN                    ISLAND   
30346                       NaN                       NaN   
30347                       NaN                       NaN   
30348                       NaN                       NaN   
30349                       NaN                      16TH   

       address_sfx_intersecting call_type disposition  beat  priority  \
30345                       NaN         T           K   515         2   
30346                       NaN      5150         CAN   623         2   
30347                       NaN         T           K   516         2   
30348                       NaN       586           O   934         4   
30349                       NaN      1131           K   521         1   

      address_number_primary_str                                       Address  
30345                        NaN                HOITT ST San Diego, California  
30346                        400  400 HOTEL CIRCLE SOUTH San Diego, California  
30347                        600             600 32ND ST San Diego, California  
30348                      13100      13100 SEA KNOLL CT San Diego, California  
30349                        NaN                    E ST San Diego, California

# Geo Location Data Google Lat and Lonf

In [111]:
def extract_lat_long_via_address(address):
    lat, lng, zipcode  = None, None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address}&key={api_key}"
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        #error
        return None, None
    try:
        #found
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
        zipcode = results['address_components'][-1]['long_name']
        
    except:
        pass
    return lat, lng, zipcode

def enrich_with_geocoding_api(row):
    column_name = 'Address'
    address_value = row[column_name]
    address_lat, address_lng, address_zip = extract_lat_long_via_address(address_value)
    row['lat'] = address_lat
    row['lng'] = address_lng
    row['zipcode'] = address_zip
    
    return row

In [118]:
Last2000 = Address_Data.tail(2000)
Last10 = Address_Data.tail(10)

In [134]:
Last10_Geo = Last10.apply(enrich_with_geocoding_api, axis=1)

In [136]:
Last10_Geo

incident_num            date_time  day_of_week  address_number_primary  \
30340  E23010032551  2023-01-23 23:19:57            2                    5900   
30341  E23010032552  2023-01-23 23:20:53            2                    1900   
30342  E23010032558  2023-01-23 23:29:24            2                       0   
30343  E23010032560  2023-01-23 23:32:57            2                    8800   
30344  E23010032561  2023-01-23 23:35:23            2                   13200   
30345  E23010032562  2023-01-23 23:35:53            2                       0   
30346  E23010032563  2023-01-23 23:38:24            2                     400   
30347  E23010032565  2023-01-23 23:39:02            2                     600   
30348  E23010032567  2023-01-23 23:44:30            2                   13100   
30349  E23010032570  2023-01-23 23:49:32            2                       0   

       address_dir_primary address_road_primary address_sfx_primary  \
30340                  NaN           UNIVERSITY                 AVE   
30341                  NaN             EL CAJON                 BLV   
30342                  NaN               GARNET                 AVE   
30343                  NaN               NAVAJO                  RD   
30344                  NaN              LUCKETT                  CT   
30345                  NaN                HOITT                  ST   
30346                  NaN   HOTEL CIRCLE SOUTH                 NaN   
30347                  NaN                 32ND                  ST   
30348                  NaN            SEA KNOLL                  CT   
30349                  NaN                    E                  ST   

       address_dir_intersecting address_road_intersecting  \
30340                       NaN                       NaN   
30341                       NaN                       NaN   
30342                       NaN                  INGRAHAM   
30343                       NaN                       NaN   
30344                       NaN                       NaN   
30345                       NaN                    ISLAND   
30346                       NaN                       NaN   
30347                       NaN                       NaN   
30348                       NaN                       NaN   
30349                       NaN                      16TH   

       address_sfx_intersecting call_type disposition  beat  priority  \
30340                       NaN         T           K   822         2   
30341                       NaN       911         CAN   624         1   
30342                       NaN      1150           K   122         2   
30343                       NaN       911         CAN   324         1   
30344                       NaN      1153           K   934         2   
30345                       NaN         T           K   515         2   
30346                       NaN      5150         CAN   623         2   
30347                       NaN         T           K   516         2   
30348                       NaN       586           O   934         4   
30349                       NaN      1131           K   521         1   

      address_number_primary_str  \
30340                       5900   
30341                       1900   
30342                        NaN   
30343                       8800   
30344                      13200   
30345                        NaN   
30346                        400   
30347                        600   
30348                      13100   
30349                        NaN   

                                            Address        lat         lng  \
30340     5900 UNIVERSITY AVE San Diego, California  32.750251 -117.070386   
30341       1900 EL CAJON BLV San Diego, California  32.755362 -117.144998   
30342              GARNET AVE San Diego, California  32.801114 -117.234831   
30343          8800 NAVAJO RD San Diego, California  32.802984 -117.008199   
30344        13200 LUCKETT CT San Diego, California  32.959768 -117.20

In [98]:
Last10.to_csv('Last10.csv')

In [119]:
Last2000_Geo = Last2000.apply(enrich_with_geocoding_api, axis=1)

In [121]:
Last2000_Geo.to_csv('Last2000_Geo.csv')

In [122]:
Last2000_Geo

incident_num            date_time  day_of_week  address_number_primary  \
28350  E23010030360  2023-01-22 11:16:00            1                    4200   
28351  E23010030361  2023-01-22 11:17:20            1                    6400   
28352  E23010030362  2023-01-22 11:18:59            1                       0   
28353  E23010030363  2023-01-22 11:19:20            1                    3500   
28354  E23010030364  2023-01-22 11:23:00            1                    1000   
...             ...                  ...          ...                     ...   
30345  E23010032562  2023-01-23 23:35:53            2                       0   
30346  E23010032563  2023-01-23 23:38:24            2                     400   
30347  E23010032565  2023-01-23 23:39:02            2                     600   
30348  E23010032567  2023-01-23 23:44:30            2                   13100   
30349  E23010032570  2023-01-23 23:49:32            2                       0   

      address_dir_primary address_road_primary address_sfx_primary  \
28350                 NaN                  ETA                  ST   
28351                 NaN        LAKE KATHLEEN                 AVE   
28352                 NaN             CAPE MAY                 AVE   
28353                 NaN               EUCLID                 AVE   
28354                 NaN        FIESTA ISLAND                  RD   
...                   ...                  ...                 ...   
30345                 NaN                HOITT                  ST   
30346                 NaN   HOTEL CIRCLE SOUTH                 NaN   
30347                 NaN                 32ND                  ST   
30348                 NaN            SEA KNOLL                  CT   
30349                 NaN                    E                  ST   

       address_dir_intersecting address_road_intersecting  \
28350                       NaN                       NaN   
28351                       NaN                       NaN   
28352                       NaN                    ABBOTT   
28353                       NaN                       NaN   
28354                       NaN                       NaN   
...                         ...                       ...   
30345                       NaN                    ISLAND   
30346                       NaN                       NaN   
30347                       NaN                       NaN   
30348                       NaN                       NaN   
30349                       NaN                      16TH   

       address_sfx_intersecting call_type disposition  beat  priority  \
28350                       NaN      459A           U   443         3   
28351                       NaN      11-7           K   324         1   
28352                       NaN       586           O   614         4   
28353                       NaN       602           K   833         2   
28354                       NaN      1151           O   123         2   
...                         ...       ...         ...   ...       ...   
30345                       NaN         T           K   515         2   
30346                       NaN      5150         CAN   623         2   
30347                       NaN         T           K   516         2   
30348                       NaN       586           O   934         4   
30349                       NaN      1131           K   521         1   

      address_number_primary_str  \
28350                       4200   
28351                       6400   
28352                        NaN   
28353                       3500   
28354                       1000   
...                          ...   
30345                        NaN   
30346                        400   
30347                        600   
30348                      13100   
30349                        NaN   

                                            Address        lat         lng  \
28350             4200 ETA ST San Diego, California  32.686686 -117.1037

# Geo Location Data Zipcode

In [128]:
locator = Nominatim(user_agent='myGeocoder')

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location_geo'] = df['Address'].apply(geocode)
df['point'] = df['location_geo'].apply(lambda loc: tuple(loc.point) if loc else (0, 0, 0))
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(),index=df.index)                                                         

In [152]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]))
        result = location.raw['address']['postcode']
    except:
        result = None
    return result


In [124]:
df = Last10.copy()

incident_num            date_time  day_of_week  address_number_primary  \
30340  E23010032551  2023-01-23 23:19:57            2                    5900   
30341  E23010032552  2023-01-23 23:20:53            2                    1900   
30342  E23010032558  2023-01-23 23:29:24            2                       0   
30343  E23010032560  2023-01-23 23:32:57            2                    8800   
30344  E23010032561  2023-01-23 23:35:23            2                   13200   
30345  E23010032562  2023-01-23 23:35:53            2                       0   
30346  E23010032563  2023-01-23 23:38:24            2                     400   
30347  E23010032565  2023-01-23 23:39:02            2                     600   
30348  E23010032567  2023-01-23 23:44:30            2                   13100   
30349  E23010032570  2023-01-23 23:49:32            2                       0   

      address_dir_primary address_road_primary address_sfx_primary  \
30340                 NaN           UNIVERSITY                 AVE   
30341                 NaN             EL CAJON                 BLV   
30342                 NaN               GARNET                 AVE   
30343                 NaN               NAVAJO                  RD   
30344                 NaN              LUCKETT                  CT   
30345                 NaN                HOITT                  ST   
30346                 NaN   HOTEL CIRCLE SOUTH                 NaN   
30347                 NaN                 32ND                  ST   
30348                 NaN            SEA KNOLL                  CT   
30349                 NaN                    E                  ST   

       address_dir_intersecting address_road_intersecting  \
30340                       NaN                       NaN   
30341                       NaN                       NaN   
30342                       NaN                  INGRAHAM   
30343                       NaN                       NaN   
30344                       NaN                       NaN   
30345                       NaN                    ISLAND   
30346                       NaN                       NaN   
30347                       NaN                       NaN   
30348                       NaN                       NaN   
30349                       NaN                      16TH   

       address_sfx_intersecting call_type disposition  beat  priority  \
30340                       NaN         T           K   822         2   
30341                       NaN       911         CAN   624         1   
30342                       NaN      1150           K   122         2   
30343                       NaN       911         CAN   324         1   
30344                       NaN      1153           K   934         2   
30345                       NaN         T           K   515         2   
30346                       NaN      5150         CAN   623         2   
30347                       NaN         T           K   516         2   
30348                       NaN       586           O   934         4   
30349                       NaN      1131           K   521         1   

      address_number_primary_str                                       Address  
30340                       5900     5900 UNIVERSITY AVE San Diego, California  
30341                       1900       1900 EL CAJON BLV San Diego, California  
30342                        NaN              GARNET AVE San Diego, California  
30343                       8800          8800 NAVAJO RD San Diego, California  
30344                      13200        13200 LUCKETT CT San Diego, California  
30345                        NaN                HOITT ST San Diego, California  
30346                        400  400 HOTEL CIRCLE SOUTH San Diego, California  
30347                        600             600 32ND ST San Diego, California  
30348                      13100      13100 SEA KNOLL CT San Diego, California  
30349                        NaN                    E ST S

In [154]:
zipcodes = Last10_Geo.apply(get_zipcode,
                            axis=1,
                            geolocator=locator,
                            lat_field='lat',
                            lon_field='lng'
                           )

In [155]:
zipcodes

30340    92115
30341    92104
30342    92109
30343    92119
30344    92130
30345    92102
30346    92103
30347    92102
30348    92130
30349    92102
dtype: object

In [ ]:
Last2000_Geo['Zipcodes'] = Last2000_Geo.apply(get_zipcode,
                            axis=1,
                            geolocator=locator,
                            lat_field='lat',
                            lon_field='lng'
                           )

In [ ]:
Last2000_Geo.head()